# Aquisição de dados no portal da ONS Dados Aberto

https://ons-aws-prod-opendata.s3.amazonaws.com/

https://registry.opendata.aws/ons-opendata-portal/

https://github.com/awslabs/open-data-registry/blob/main/datasets/ons-opendata-portal.yaml#L17

In [ ]:
import requests
import xml.etree.ElementTree as ET

# Lista todos os arquivos do bucket


def listar_arquivos():
    url = "https://ons-aws-prod-opendata.s3.amazonaws.com"
    params = {"list-type": "2", "prefix": "dataset/"}
    arquivos = []

    while True:
        resp = requests.get(url, params=params)
        root = ET.fromstring(resp.content)
        for item in root.findall(".//{*}Contents"):
            arquivos.append(item.find("{*}Key").text)

        next_token = root.find(".//{*}NextContinuationToken")
        if next_token is None:
            break
        params["continuation-token"] = next_token.text

    return arquivos

# Lista nomes únicos de datasets


def listar_datasets_unicos(arquivos):
    return sorted(set(a.split("/")[1] for a in arquivos if a.startswith("dataset/")))

# Lista arquivos dentro de um dataset escolhido


def listar_arquivos_dataset(arquivos, dataset):
    return [a for a in arquivos if a.startswith(f"dataset/{dataset}/") and not a.endswith("/")]


# ---- Uso ----
arquivos = listar_arquivos()
datasets = listar_datasets_unicos(arquivos)

# Exibe os datasets
print("Datasets disponíveis:")
for i, d in enumerate(datasets):
    print(f"{i + 1}. {d}")

# Seleciona um dataset
escolha = int(input("\nEscolha um número: ")) - 1
selecionado = datasets[escolha]

# Exibe os arquivos do dataset
print(f"\nArquivos em '{selecionado}':")
for arq in listar_arquivos_dataset(arquivos, selecionado):
    print("-", arq.split("/")[-1])

dataset/
dataset/bacia_contorno/
dataset/bacia_contorno/Bacias_Hidrograficas_SIN.zip
dataset/balanco_dessem_detalhe/
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_23.csv
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_23.parquet
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_23.xlsx
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_24.csv
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_24.parquet
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_24.xlsx
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_25.csv
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_25.parquet
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_25.xlsx
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_26.csv
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_26.parquet
dataset/balanco_dessem_detalhe/BALANCO_DESSEM_DETALHE_2025_05_26.xlsx
dataset/balanco_dessem_detalhe/BALA